In [11]:
# sorry for the variable names
import nltk
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

# load mo yung csv na dinownload mo sa google forms, ibahin mo nalang yung path pag di gumana sayo
df1 = pd.read_csv("Sentiment Analysis on DLSU-D Tuition Fee Increase.csv")

# ignore the first two columns ("Timestamp" and "Name")
question_columns = list(df1.columns.values[2:])

# the questions used in the survey
print("Questions")
question_num = 1
for question in question_columns:
    print(f"{question_num}. {question}")
    question_num += 1
print("\n")

# replace the column names with a shorter one for convenience
question_num = 1
for column in df1:
    if column == "Timestamp" or column == "Name":
        pass
    else:
        df1.rename(columns={column:f"Q{question_num}"}, inplace=True)
        question_num += 1

question_columns_processed = list(df1.columns.values[2:])

def process_dataframe(df):
    """Gets the polarity scores and other data about an answer."""


    df.dropna(inplace=True)
    blanks = []
    """
    for i, lb, rv in df.itertuples():
        if type(rv) == str:
            if rv.isspace():
                blanks.append(i)

        df.drop(blanks, inplace=True)
    """
    # df.iloc[:,0] means the first column, regardless of what dataframe you're trying to process
    df["polarity_scores"] = df.iloc[:,0].apply(lambda review: sia.polarity_scores(review))
    df["compound"] = df["polarity_scores"].apply(lambda score_dict: score_dict["compound"])
    df["compound_score"] = df["compound"].apply(lambda score: "pos" if score >= 0 else "neg")

    # hula lang tong part na to
    df["label"] = df["polarity_scores"].apply(lambda score_dict: "pos" if score_dict["pos"] >= score_dict["neg"] else "neg")

    # most likely wrong
    # df["label"] = df["polarity_scores"].apply(lambda score_dict: "pos" if score_dict["pos"] or score_dict["neu"] > score_dict["neg"] else "neg")

def process_dataframe_metrics(df):
    """Does the matrix thing."""
    # doon sa example, yung dataset na ginamit nya may column na nakapangalan na
    # "label". kapag dinownload mo yung csv doon sa survey natin, walang column na ganon
    # yung pangalan. so baka literal na mano-mano natin lalagyan yung bawat isang answer
    # doon sa dataset, kung ang sagot ba nila ay positive o negative. di ko rin alam eh.
    # in the meantime, walang sense tong function na to, kaya wag muna galawin.
    print("Accuracy Score:", accuracy_score(df["label"], df["compound_score"]))
    print("Classification Report:\n", classification_report(df["label"], df["compound_score"]))
    print("Confusion Matrix:\n", confusion_matrix(df["label"], df["compound_score"]))

    # garbage
    """
    print("Accuracy Score:", accuracy_score(df["compound_score"], df["compound_score"]))
    print("Classification Report:\n", classification_report(df["compound_score"], df["compound_score"]))
    print("Confusion Matrix:\n", confusion_matrix(df["compound_score"], df["compound_score"]))
    """

def generate_question_df(df):
    """Gets all answers for each question, and then processes each answer."""
    question_df = []
    question_num = 1
    for column in question_columns_processed:
        question_df.append(pd.DataFrame({f"Q{question_num}": list(df.loc[:, f"Q{question_num}"])}))
        question_num += 1
    for df in question_df:
        process_dataframe(df)
    return question_df

# use 0 for the first question, 9 for last
EXAMPLE = 4

# get list of questions with their corresponding responses
question_df_list = generate_question_df(df1)

# process 1 dataframe related to that question to get the metrics
process_dataframe_metrics(question_df_list[EXAMPLE])

# print the question being asked
print("\nQuestion:", question_columns[EXAMPLE])

# print all responses to that question and its corresponding data
question_df_list[EXAMPLE].head(99999)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Questions
1. Are you satisfied with your course? Why do you feel like it?
2. Do you think the course that you chose is worth it? Why?
3. Is the tuition fee increase justifiable? Give insight on how it affects you.
4. Do you think that they do this to catch up with other institutions? If so, why do you think that is?
5. Are you satisfied with the materials and equipment that the university provides you? Elaborate and provide feedback, if able.
6. Is the tuition fee increase manageable? How do you think to manage it?
7. How will you plan for future tuition fee increases?
8. What additional expenses should you expect to pay for besides the tuition fee?
9. What external factors do you think affects how you manage the tuition fee?
10. How should students cope with future tuition fee increases?


Accuracy Score: 1.0
Classification Report:
               precision    recall  f1-score   support

         neg       1.00      1.00      1.00        11
         pos       1.00      1.00      1.00  

,Q5,polarity_scores,compound,compound_score,label
0,"Yes, because i do think all we need is in the ...","{'neg': 0.0, 'neu': 0.944, 'pos': 0.056, 'comp...",0.2144,pos,pos
1,I am not satisfied because the equipment that ...,"{'neg': 0.076, 'neu': 0.751, 'pos': 0.173, 'co...",0.4530,pos,pos
2,"No, I am not satisfied with the materials and ...","{'neg': 0.249, 'neu': 0.751, 'pos': 0.0, 'comp...",-0.7865,neg,neg
3,No. No I am not satisfied.,"{'neg': 0.771, 'neu': 0.229, 'pos': 0.0, 'comp...",-0.6939,neg,neg
4,"No, because it has gotten less and worse overt...","{'neg': 0.418, 'neu': 0.582, 'pos': 0.0, 'comp...",-0.6151,neg,neg
5,The schoolbook material was a good equipment a...,"{'neg': 0.0, 'neu': 0.797, 'pos': 0.203, 'comp...",0.8932,pos,pos
6,Not very much. I would prefer to have faster a...,"{'neg': 0.0, 'neu': 0.829, 'pos': 0.171, 'comp...",0.4754,pos,pos
7,No. We dont have access to the supposed labora...,"{'neg': 0.196, 'neu': 0.804, 'pos': 0.0, 'comp...",-0.2960,neg,neg
8,Yes,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.4019,pos,pos
9,No. There are numerous resources that are avai...,"{'neg': 0.058, 'neu': 0.863, 'pos': 0.079, 'co...",0.1531,pos,pos


# mali tong part na to ↓↓↓

In [ ]:
import nltk
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

sia = SentimentIntensityAnalyzer()

df2 = pd.read_csv("ds/Sentiment Analysis on DLSU-D Tuition Fee Increase.csv")

question_columns = list(df.columns.values[2:])

# the questions used in the survey
print("Questions")
question_num = 1
for question in question_columns:
    print(f"{question_num}. {question}")
    question_num += 1
print("\n")

# replace column names with a shorter name for convenience
question_num = 1
for column in df2:
    if column == "Timestamp" or column == "Name":
        pass
    else:
        df.rename(columns={column:f"Q{question_num}"}, inplace=True)
        question_num += 1

question_columns_processed = list(df2.columns.values[2:])

print("Accuracy Score:", accuracy_score(question_columns_processed))
# print("Classification Report:\n", classification_report(df["label"], df["compound_score"]))
# print("Confusion Matrix:\n", confusion_matrix(df["label"], df["compound_score"]))

Questions
1. Q1
2. Q2
3. Q3
4. Q4
5. Q5
6. Q6
7. Q7
8. Q8
9. Q9
10. Q10




TypeError: missing a required argument: 'y_pred'

In [ ]:
answers = [
    "Neutral. I believe that we have both pros and cons on choosing the course.",
    "I feel that it is lacking, with the increase of the tuition fee and comparing it to the facilities offered to us. It is not fair that we have such high tuition fee yet have low-quality facilities and computers. I am not satisfied, I feel that it is not good.",
    "Yes, I am satisfied. It is very fun learning new and different stuff relating to computers.",
    "Yes! Because I love what I do.",
    "No. Because I do not have any access to laboratory room and equipment.",
    "This product sucks. I want my fucking money back!"
]

for answer in answers:
    print(answer[:20], "...", sia.polarity_scores(answer))

Neutral. I believe t ... {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
I feel that it is la ... {'neg': 0.129, 'neu': 0.827, 'pos': 0.044, 'compound': -0.5267}
Yes, I am satisfied. ... {'neg': 0.0, 'neu': 0.569, 'pos': 0.431, 'compound': 0.8439}
Yes! Because I love  ... {'neg': 0.0, 'neu': 0.294, 'pos': 0.706, 'compound': 0.8016}
No. Because I do not ... {'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'compound': -0.296}
This product sucks.  ... {'neg': 0.255, 'neu': 0.612, 'pos': 0.133, 'compound': -0.296}
